# Black-Scholes formula european call

In [1]:
import numpy as np
from scipy.stats import norm

def black_scholes_formula(S0, T, k, sigma, r):
    d1 = (np.log(S0/k)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    Nd1 = norm.cdf(d1, loc=0, scale=1)
    Nd2 = norm.cdf(d2, loc=0, scale=1)
    C = S0*Nd1 - k*np.exp(-r*T)*Nd2
    return C
    


In [42]:
black_scholes_formula(100,5,100,0.3,0.05)

35.95780653844323

# Black-Scholes formula european Put

In [3]:
def BSPut(S0,T,k,sigma,r):
    d1 = (np.log(S0/k)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    Nd1 = norm.cdf(-d1)
    Nd2 = norm.cdf(-d2)
    P = k*np.exp(-r*T)*Nd2 - S0*Nd1
    return P

In [25]:
BSPut(100,5,100,0.3,0.05)

13.837884845583734

# Monte Carlo method european option

In [11]:
def MonteCarlo(Option,N,S0,T,k,sigma,r,alpha=0.05):
    # stocks simulation:
    g = np.random.randn(N)
    BT = g*np.sqrt(T)
    ST = S0*np.exp((r-0.5*sigma**2)*T+sigma*BT)
    # payoffs
    if(Option == 'C'):
        payoff = np.maximum(0,ST-k)
    if (Option == 'P'):
        payoff = np.maximum(0,k-ST)
    # discounted payoff
    disc_payoff = np.exp(-r*T)*payoff
    # montecarlo estimator 
    MC_est = disc_payoff.mean()
    # confidence intervall
    stddev = np.std(disc_payoff,ddof=1)
    z = norm.ppf(1-alpha/2)
    inf = MC_est - z*stddev/np.sqrt(N)
    sup = MC_est + z*stddev/np.sqrt(N)
    return(inf,MC_est,sup,sup-inf)

In [31]:
MonteCarlo('P',1000000,100,5,100,0.3,0.05)

(13.799300501822033,
 13.835353665159033,
 13.871406828496033,
 0.036053163336999816)

# CRR European (matrix version)

In [8]:
import numpy as np

def CRR_matrix_eu(Option,N,S0,T,k,sigma,r):
    # setting
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    q = (np.exp(r*dt)-d)/(u-d)
    disc = np.exp(-r*dt)
    # matrix and price at maturity
    V = np.zeros((N+1,N+1))
    J = np.arange(N+1) # vettore 0,1,2,..,N
    ST = S0*(u**(N-J))*d**J  #price at maturity
    # payoff
    if Option == 'C':
        payoff = np.maximum(0,ST-k)
    if Option == 'P':
        payoff = np.maximum(0,k-ST)
    V[:,N] = payoff
    # backward induction
    for i in range(N-1,-1,-1):
        V[0:i+1,i]=disc*(q*V[0:i+1,i+1]+(1-q)*V[1:i+2,i+1])
    return V[0,0]


In [68]:
CRR_matrix_eu('P',1000,100,5,100,0.3,0.05)

13.831760235421068

# CRR European (vector version)

In [7]:
import numpy as np

def CRR_vector_eu(Option,N,S0,T,k,sigma,r):
    #setting
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    q = (np.exp(r*dt)-d)/(u-d)
    disc = np.exp(-r*dt)
    # prices at maturity
    J = np.arange(N+1)  # 0,1,...,N
    ST = S0*(u**(N-J))*d**J
    if Option == 'C':
        V = np.maximum(0,ST-k) #payoff
    if Option == 'P':
        V = np.maximum(0,k-ST) 
    # backward induction
    for i in range(N-1,-1,-1): #  N-1,...,1,0
        V[0:i+1] = disc*(q*V[0:i+1]+(1-q)*V[1:i+2])
    return V[0]

In [64]:
CRR_vector_eu('P',1000,100,5,100,0.3,0.05)

13.831760235421068

# CRR American (vector version)

In [12]:
import numpy as np

def CRR_American(Option,N,S0,T,k,sigma,r):
    # setting
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    q = (np.exp(r*dt)-d)/(u-d)
    disc = np.exp(-r*dt)
    # Stocks prices
    S = np.zeros((N+1,N+1))
    S[0,0] = S0
    for n in range(1,N+1):
        j = np.arange(n+1)
        S[0:n+1,n] = S0*(u**(n-j))*d**j
    # intrinsic values
    if Option == 'C':
        IN_V = np.maximum(S-k,0)
    if Option == 'P':
        IN_V = np.maximum(k-S,0)
    # maturity payoff
    V = IN_V[:,N]
    # backward induction
    for i in range(N-1,-1,-1): # N-1,...,1,0
        viva = disc*(q*V[0:i+1]+(1-q)*V[1:i+2])
        V = np.maximum(IN_V[0:i+1,i],viva)

    return V[0]


In [35]:
CRR_American('C',1000,100,5,100,0.3,0.05)

35.95168192828498

# CRR American with exercise matrix

In [63]:
import numpy as np

def CRR_American_exercise(Option,N,S0,T,k,sigma,r):
    # setting
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    q = (np.exp(r*dt)-d)/(u-d)         
    disc = np.exp(-r*dt)

    # intrinsec values
    S = np.zeros((N+1,N+1))  
    S[0,0]=S0                
    for n in range(1,N+1):    
        j = np.arange(n+1)  
        S[0:n+1,n] = S0*(u**(n-j))*(d**j)
    if Option == 'C':
        InV = np.maximum(S-k,0)  
    if Option == 'P':
        InV = np.maximum(k-S,0)
        
    # exercise matrix
    exercise = np.zeros_like(S,dtype = bool) 
    exercise[:,N] = (InV[:,N]>0)   # at maturity exercise is true for positive payoff
    # backward induction
    V = np.zeros_like(S)
    V[:,N] = InV[:,N]   
    for n in range(N-1,-1,-1):   # da N-1 a 0 compresi
        continuation = disc*(q*V[0:n+1,n+1]+(1-q)*V[1:n+2,n+1])   
        V[0:n+1,n] = np.maximum(continuation,InV[0:n+1,n])
        early_exercise = (InV[0:n+1,n] > continuation)
        exercise[0:n+1,n] = early_exercise

    return V, exercise
    

In [65]:
print(CRR_American_exercise('P',3,100,1,100,0.3,0.05)[0])
print(CRR_American_exercise('P',3,100,1,100,0.3,0.05)[1])

[[10.67948975  3.76777348  0.          0.        ]
 [ 0.         18.09576232  7.74084854  0.        ]
 [ 0.          0.         29.27776478 15.90348686]
 [ 0.          0.          0.         40.52506616]]
[[False False False False]
 [False False False False]
 [False False  True  True]
 [False False False  True]]


In this little exemple we can observe the triangular ercise region.

# Testing MonteCarlo method accuracy 
We consider the values S0=100, T=5, k=100, sigma=0.3, r=0.05.\
The function is executed 50 times for every N = 100, 1000, 10000, 100000 to estimate the price and the confidence interval, and their mean values are then computed.

In [61]:
# 100 paths
N = 100
results = []
for n in range(100):
    v = MonteCarlo('C',N,100,5,100,0.3,0.05)
    results.append(v)
tab = np.array(results)
stat = tab.mean(axis=0)
real_value = black_scholes_formula(100,5,100,0.3,0.05)
print('MC estimate:',stat[1].round(4))
print('Interval width:',stat[3].round(4))
print('Error:',np.abs(real_value-stat[1]).round(4))

MC estimate: 35.6343
Interval width: 25.248
Error: 0.3235


In [69]:
# 1000 paths
N = 1000
results = []
for n in range(100):
    v = MonteCarlo('C',N,100,5,100,0.3,0.05)
    results.append(v)
tab = np.array(results)
stat = tab.mean(axis=0)
real_value = black_scholes_formula(100,5,100,0.3,0.05)
print('MC estimate:',stat[1].round(4))
print('Interval width:',stat[3].round(4))
print('Error:',np.abs(real_value-stat[1]).round(4))

MC estimate: 35.7803
Interval width: 8.0673
Error: 0.1775


In [75]:
# 10000 paths
N = 10000
results = []
for n in range(100):
    v = MonteCarlo('C',N,100,5,100,0.3,0.05)
    results.append(v)
tab = np.array(results)
stat = tab.mean(axis=0)
real_value = black_scholes_formula(100,5,100,0.3,0.05)
print('MC estimate:',stat[1].round(4))
print('Interval width:',stat[3].round(4))
print('Error:',np.abs(real_value-stat[1]).round(4))

MC estimate: 36.0352
Interval width: 2.6017
Error: 0.0774


In [77]:
# 100000 paths
N = 100000
results = []
for n in range(100):
    v = MonteCarlo('C',N,100,5,100,0.3,0.05)
    results.append(v)
tab = np.array(results)
stat = tab.mean(axis=0)
real_value = black_scholes_formula(100,5,100,0.3,0.05)
print('MC estimate:',stat[1].round(4))
print('Interval width:',stat[3].round(4))
print('Error:',np.abs(real_value-stat[1]).round(4))

MC estimate: 35.939
Interval width: 0.817
Error: 0.0188


# Testing Monte Carlo method time performance

In [83]:
import timeit
import numpy as np
from scipy.stats import norm

params = {     # test parameters
    "Option": 'C',
    "N": 100,
    "S0": 100,
    "T": 5.0,
    "k": 100,
    "sigma": 0.3,
    "r": 0.05
}

def run_mc():
    MonteCarlo(**params)

execution_time = timeit.timeit(run_mc, number=100)

print(f"Average execution time: {execution_time / 100:.6f} seconds")


Average execution time: 0.000558 seconds


In [85]:
import timeit
import numpy as np
from scipy.stats import norm

params = {     # test parameters
    "Option": 'C',
    "N": 1000,
    "S0": 100,
    "T": 5.0,
    "k": 100,
    "sigma": 0.3,
    "r": 0.05
}

def run_mc():
    MonteCarlo(**params)

execution_time = timeit.timeit(run_mc, number=100)

print(f"Average execution time: {execution_time / 100:.6f} seconds")

Average execution time: 0.000728 seconds


In [87]:
import timeit
import numpy as np
from scipy.stats import norm

params = {     # test parameters
    "Option": 'C',
    "N": 10000,
    "S0": 100,
    "T": 5.0,
    "k": 100,
    "sigma": 0.3,
    "r": 0.05
}

def run_mc():
    MonteCarlo(**params)

execution_time = timeit.timeit(run_mc, number=100)

print(f"Average execution time: {execution_time / 100:.6f} seconds")

Average execution time: 0.001108 seconds


In [89]:
import timeit
import numpy as np
from scipy.stats import norm

params = {     # test parameters
    "Option": 'C',
    "N": 100000,
    "S0": 100,
    "T": 5.0,
    "k": 100,
    "sigma": 0.3,
    "r": 0.05
}

def run_mc():
    MonteCarlo(**params)

execution_time = timeit.timeit(run_mc, number=100)

print(f"Average execution time: {execution_time / 100:.6f} seconds")

Average execution time: 0.004642 seconds
